<a href="https://colab.research.google.com/github/lstival/mo826/blob/main/P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade P2 - Fabrício Ferreira da Silva RA: 231900 e Leandro Stival RA: 263013

##Importar os dados dos pacientes

O primeiro passo foi importar a biblioteca pandas, que é usada para manipular dados, e armazenar os arquivos que foram fornecidos que estão no formato CSV em variáveis, usando o comando read_CSV e colocando o caminho do arquivo desejado.

In [1]:
#Ignorando alguns warnings
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
import pandas as pd
import numpy as np
#Scenario 2
encounters = pd.read_csv('https://raw.githubusercontent.com/santanche/lab2learn/master/data/synthea/scenario02/csv/encounters.csv')
patients = pd.read_csv('https://raw.githubusercontent.com/santanche/lab2learn/master/data/synthea/scenario02/csv/patients.csv')
# conditions = pd.read_csv('https://raw.githubusercontent.com/santanche/lab2learn/master/data/synthea/scenario02/csv/conditions.csv')
# procedures = pd.read_csv('https://raw.githubusercontent.com/santanche/lab2learn/master/data/synthea/scenario02/csv/procedures.csv')
print(np.shape(encounters))
print(np.shape(patients))

(55285, 15)
(1121, 25)


In [3]:
encounters_death = encounters[(encounters['DESCRIPTION']=='Death Certification')]
encounters_death.drop_duplicates(subset= "Id", keep= "first", inplace=True, ignore_index=True)
# print(encounters_death)
death_variants = encounters_death['REASONDESCRIPTION'].value_counts()
# print(death_variants)
# death_variants.to_csv('causas_morte.csv')

In [4]:
patients_death = patients.dropna(subset=['DEATHDATE'])
patients_death = patients[patients['DEATHDATE'].notna()]
patients_death.drop_duplicates(subset= "Id", keep= "first", inplace=True, ignore_index=True)
# print(patients_death)

## Fatiando a base de dados
Selecionado o top 10 doenças mais letais como foco do estudo, selecionando os pacientes que possuem ao menos um encontro com ela.

In [5]:
#Save the name of the diases that will be considered
target_diases = death_variants[:10].index.values
target_diases

array(['Sudden Cardiac Death', 'Myocardial Infarction',
       'Chronic congestive heart failure (disorder)',
       'Acute myeloid leukemia  disease (disorder)', 'Bullet wound',
       'Cardiac Arrest', 'Pulmonary emphysema (disorder)',
       'Concussion injury of brain', 'Pneumonia', 'Stroke'], dtype=object)

In [6]:
#Encounter where the description is one of the target diases
target_encounters = encounters[encounters['REASONDESCRIPTION'].apply(lambda x: x in target_diases)]
# target_encounters.head()
print(f"Existem {target_encounters.index.size} encontros com os diagnosticos \
selecionados.")

Existem 1493 encontros com os diagnosticos selecionados.


In [7]:
#Pacientes com ao menos uma vez tiveram registrado com essa doença
target_patients_ids = target_encounters['PATIENT'].drop_duplicates().values
target_patients = patients[patients['Id'].apply(lambda x: x in target_patients_ids)]
print(f"Existem {target_patients.index.size} pacientes que ao menos uma vez \
tiveram a ocorrência desse diagnostico.")

Existem 235 pacientes que ao menos uma vez tiveram a ocorrência desse diagnostico.


## Dataframe com os dados de interesse do paciente

In [8]:
#Dataframe com os dados que serão utilizados para ML
df_interest_data = pd.DataFrame()
#Inserindo o Id do paciente ao Dataframe
df_interest_data['Id'] = target_patients.Id.values

## Selecionado e tratando as variáveis de interesse dos pacientes

In [9]:
def convert_2_datetime(column):
  return pd.to_datetime(column).dt.tz_localize(None)

In [10]:
target_encounters['START'] = convert_2_datetime(target_encounters['START'])
target_patients['BIRTHDATE'] = convert_2_datetime(target_patients['BIRTHDATE'])

### Cálculando a idade

In [11]:
start_encounter = convert_2_datetime(target_encounters.drop_duplicates(subset= "PATIENT", keep= "last")["START"])

In [12]:
patient_ages = (start_encounter.values - target_patients['BIRTHDATE'].values).astype('timedelta64[Y]') / np.timedelta64(1, 'Y')

In [13]:
#Inserindo a idade dos pacientes ao dataframe de interesse
df_interest_data['age'] = patient_ages

In [14]:
df_interest_data['lat'] = target_patients["LAT"]
df_interest_data['lon'] = target_patients["LON"]

In [15]:
death_dates = convert_2_datetime(target_patients['DEATHDATE'].fillna(0)).values

## Criando variável alvo

In [16]:
days_to_target = 15

In [17]:
df_interest_data["target"] = (start_encounter.values - death_dates).astype('timedelta64[D]') / np.timedelta64(1, 'D') < days_to_target

In [18]:
_, deaths = np.unique(df_interest_data["target"], return_counts=True)

In [19]:
df_interest_data['target'] = df_interest_data['target'].apply(lambda x: 1 if x is True else 0)

In [20]:
print(f"O conjunto possui {deaths[0]} pacientes que não morreram em \
{days_to_target} dias após o evento, e {deaths[1]} mortes antes dos {days_to_target} dias.")

O conjunto possui 146 pacientes que não morreram em 15 dias após o evento, e 89 mortes antes dos 15 dias.


## Prevendo se o paciente irá morrer

In [21]:
#Modelo de regressão logística
from sklearn.linear_model import LogisticRegression

#Criando treino e teste
from sklearn.model_selection import train_test_split

#Avalição do modelo
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

In [22]:
df_interest_data.dropna(inplace=True)
train, test = train_test_split(df_interest_data, train_size=0.7, stratify=df_interest_data['target'], random_state=2022)

In [23]:
def predict_values(cls, X, y):
  """
  Função responsável por realizar a predição de um modelo, e retorna
  a accurácia, uma lista com a matriz da confusão e accurácia balanceada.
  """
  predicts = cls.predict(X)
  score = cls.score(X, y)
  tn, fp, fn, tp = confusion_matrix(y, predicts).ravel()
  bl_acc = balanced_accuracy_score(y, predicts)

  return round(score, 2), [tn, fp, fn, tp], round(bl_acc, 2)

In [24]:
X = train[['age','lat','lon']].values
X = train[['age']].values
y = train['target'].values

In [25]:
clf = LogisticRegression(random_state=2022).fit(X, y)

## Testando o modelo

In [26]:
# X_test = test[['age','lat','lon']].values
X_test = test[['age']].values
y_test = test['target'].values

In [27]:
acc, confusion, bl_acc = predict_values(clf, X, y)

In [28]:
print(f"Acurácia do classificador é {acc} e a balanceada é {bl_acc}")

Acurácia do classificador é 0.69 e a balanceada é 0.59


In [29]:
#tn, fp, fn, tp
np.reshape(confusion, [2,2])

array([[19,  2],
       [ 8,  3]])

## Outro classificador

In [30]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [31]:
xgb_cl = xgb.XGBClassifier()

In [32]:
xgb_cl.fit(X, y)

XGBClassifier()

In [33]:
acc, confusion, bl_acc = predict_values(xgb_cl, X_test, y_test)

print(f"Acurácia do classificador é {acc} e a balanceada é {bl_acc}")

#tn, fp, fn, tp
np.reshape(confusion, [2,2])

Acurácia do classificador é 0.8 e a balanceada é 0.7


array([[10,  0],
       [ 3,  2]])

# Conjunto de dados 01

In [34]:
encounters1 = pd.read_csv('https://raw.githubusercontent.com/santanche/lab2learn/master/data/synthea/scenario01/csv/encounters.csv')
patients1 = pd.read_csv('https://raw.githubusercontent.com/santanche/lab2learn/master/data/synthea/scenario01/csv/patients.csv')

print(np.shape(encounters1))
print(np.shape(patients1))

(59481, 15)
(1174, 25)


In [35]:
encounters_death1 = encounters1[(encounters1['DESCRIPTION']=='Death Certification')]
encounters_death1.drop_duplicates(subset= "Id", keep= "first", inplace=True, ignore_index=True)
# print(encounters_death)
death_variants1 = encounters_death1['REASONDESCRIPTION'].value_counts()
# print(death_variants)
# death_variants.to_csv('causas_morte_01.csv')

In [36]:
patients_death1 = patients1.dropna(subset=['DEATHDATE'])
patients_death1 = patients1[patients1['DEATHDATE'].notna()]
patients_death1.drop_duplicates(subset= "Id", keep= "first", inplace=True, ignore_index=True)
# print(patients_death1)

## Fatiando a base de dados
Selecionado o top 10 doenças mais letais como foco do estudo, selecionando os pacientes que possuem ao menos um encontro com ela.

In [37]:
#Save the name of the diases that will be considered
target_diases1 = death_variants1[:10].index.values
target_diases1

array(['Acute myeloid leukemia  disease (disorder)',
       'Myocardial Infarction',
       'Chronic congestive heart failure (disorder)', 'Pneumonia',
       'Sudden Cardiac Death', 'Non-small cell lung cancer (disorder)',
       'Stroke', 'Bullet wound', 'COVID-19',
       'Chronic obstructive bronchitis (disorder)'], dtype=object)

In [38]:
#Encounter where the description is one of the target diases
target_encounters1 = encounters1[encounters1['REASONDESCRIPTION'].apply(lambda x: x in target_diases)]
# target_encounters.head()
print(f"Existem {target_encounters1.index.size} encontros com os diagnosticos \
selecionados.")

Existem 1032 encontros com os diagnosticos selecionados.


In [39]:
#Pacientes com ao menos uma vez tiveram registrado com essa doença
target_patients_ids1 = target_encounters1['PATIENT'].drop_duplicates().values
target_patients1 = patients1[patients1['Id'].apply(lambda x: x in target_patients_ids1)]
print(f"Existem {target_patients1.index.size} pacientes que ao menos uma vez \
tiveram a ocorrência desse diagnostico.")

Existem 267 pacientes que ao menos uma vez tiveram a ocorrência desse diagnostico.


## Dataframe com os dados de interesse do paciente

In [40]:
#Dataframe com os dados que serão utilizados para ML
df_interest_data1 = pd.DataFrame()
#Inserindo o Id do paciente ao Dataframe
df_interest_data1['Id'] = target_patients1.Id.values

## Selecionado e tratando as variáveis de interesse dos pacientes

In [41]:
def convert_2_datetime(column):
  return pd.to_datetime(column).dt.tz_localize(None)

In [42]:
target_encounters1['START'] = convert_2_datetime(target_encounters1['START'])
target_patients1['BIRTHDATE'] = convert_2_datetime(target_patients1['BIRTHDATE'])

### Cálculando a idade

In [43]:
start_encounter1 = convert_2_datetime(target_encounters1.drop_duplicates(subset= "PATIENT", keep= "last")["START"])

In [44]:
patient_ages1 = (start_encounter1.values - target_patients1['BIRTHDATE'].values).astype('timedelta64[Y]') / np.timedelta64(1, 'Y')

In [45]:
#Inserindo a idade dos pacientes ao dataframe de interesse
df_interest_data1['age'] = patient_ages1

df_interest_data1['lat'] = target_patients1["LAT"]
df_interest_data1['lon'] = target_patients1["LON"]

In [46]:
death_dates1 = convert_2_datetime(target_patients1['DEATHDATE'].fillna(0)).values

## Criando variável alvo

In [47]:
days_to_target1 = 15

In [48]:
df_interest_data1["target"] = (start_encounter1.values - death_dates1).astype('timedelta64[D]') / np.timedelta64(1, 'D') < days_to_target1

In [49]:
_, deaths1 = np.unique(df_interest_data1["target"], return_counts=True)

In [50]:
df_interest_data1['target'] = df_interest_data1['target'].apply(lambda x: 1 if x is True else 0)

In [51]:
print(f"O conjunto possui {deaths1[0]} pacientes que não morreram em \
{days_to_target1} dias após o evento, e {deaths1[1]} mortes antes dos {days_to_target1} dias.")

O conjunto possui 167 pacientes que não morreram em 15 dias após o evento, e 100 mortes antes dos 15 dias.


## Prevendo se o paciente irá morrer

In [52]:
#Modelo de regressão logística
from sklearn.linear_model import LogisticRegression

#Criando treino e teste
from sklearn.model_selection import train_test_split

#Avalição do modelo
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

In [53]:
df_interest_data1.dropna(inplace=True)

In [54]:
train1, test1 = train_test_split(df_interest_data1, train_size=0.7, stratify=df_interest_data1['target'], random_state=2022)

In [55]:
# X = train[['age','lat','lon']].values
X1 = train1[['age']].values
y1 = train1['target'].values

In [56]:
clf1 = LogisticRegression(random_state=2022).fit(X1, y1)

## Dados de teste

In [57]:
# X_test = test[['age','lat','lon']].values
X_test1 = test1[['age']].values
y_test1 = test1['target'].values

In [58]:
acc1, confusion1, bl_acc1 = predict_values(clf1, X_test1, y_test1)

print(f"Acurácia do classificador é {acc1} e a balanceada é {bl_acc1}")

#tn, fp, fn, tp
np.reshape(confusion1, [2,2])

Acurácia do classificador é 0.86 e a balanceada é 0.82


array([[13,  1],
       [ 2,  5]])

## Outro classificador

In [59]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [60]:
xgb_cl1 = xgb.XGBClassifier()
xgb_cl1.fit(X1, y1)

XGBClassifier()

In [61]:
acc1, confusion1, bl_acc1 = predict_values(xgb_cl1, X_test1, y_test1)

print(f"Acurácia do classificador é {acc1} e a balanceada é {bl_acc1}")

#tn, fp, fn, tp
np.reshape(confusion1, [2,2])

Acurácia do classificador é 0.71 e a balanceada é 0.71


array([[10,  4],
       [ 2,  5]])

## Intercambinado classificadores 
#### (treinado em uma base testado na outra)

## Treinado em 2 e testado em 1

In [62]:
acc1, confusion1, bl_acc1 = predict_values(xgb_cl, X_test1, y_test1)

print(f"Acurácia do classificador é {acc1} e a balanceada é {bl_acc1}")

Acurácia do classificador é 0.81 e a balanceada é 0.79


In [63]:
acc1, confusion1, bl_acc1 = predict_values(clf, X_test1, y_test1)

print(f"Acurácia do classificador é {acc1} e a balanceada é {bl_acc1}")

Acurácia do classificador é 0.81 e a balanceada é 0.75


## Treinado em 1 e testado em 2

In [64]:
acc1, confusion1, bl_acc1 = predict_values(xgb_cl1, X_test, y_test)

print(f"Acurácia do classificador é {acc1} e a balanceada é {bl_acc1}")

Acurácia do classificador é 0.67 e a balanceada é 0.6


In [65]:
acc1, confusion1, bl_acc1 = predict_values(clf1, X_test, y_test)

print(f"Acurácia do classificador é {acc1} e a balanceada é {bl_acc1}")

Acurácia do classificador é 0.8 e a balanceada é 0.7
